In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import misc.autointerpolation

# 1. Load and Preprocess Data

In [2]:
# path = "E:/Projects/SEACAR_WQ_Pilot/"
# gis_path = path+'GIS_data/'

path = "D:/Water_Quailty/"
gis_path = path+'GIS_data/'

In [3]:
# Reload modules in external .py files after editing.
dfAll = pd.read_csv(path + "all_0214.csv").drop(columns=['Unnamed: 0','Unnamed: 0.1','RowID','ValueQualifier'])
# Convert SampleDate froms str to date
dfAll['SampleDate'] = pd.to_datetime(dfAll['SampleDate']).dt.date

C:\Users\xiang11\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
dfSeason = pd.read_csv(path + "OEATUSF_Geospatial_TempSeasons_update_0502.csv")
dfSeason = dfSeason.dropna(subset=["s_start","s_end"]).reset_index(drop=True)

dfSeason['start_date'] = pd.to_datetime(dfSeason['s_start'])
dfSeason["end_date"] = pd.to_datetime(dfSeason["s_end"])
dfSeason["date_range"] = dfSeason["end_date"] - dfSeason["start_date"]
dfSeason["daterange"] = dfSeason.apply(lambda x: int(str(x["date_range"]).split(" ")[0]),axis=1)
dfSeason = dfSeason[dfSeason["daterange"]>0].drop(columns="daterange").reset_index(drop=True)
data = dfSeason.assign(**{'Total Nitrogen': np.nan, 'Dissolved Oxygen': np.nan, 'Salinity': np.nan, 'Secchi Depth': np.nan, 'Turbidity': np.nan})

In [5]:
# Convert full MA names to short names
dictArea     = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'GTM Reserve','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}
# Convert full MA names to MA name in ORCP_Managed_Areas_Oct2021
dictArea2    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'Gasparilla Sound-Charlotte Harbor','Big Bend Seagrasses Aquatic Preserve':'Big Bend Seagrasses',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'Guana Tolomato Matanzas NERR','Estero Bay Aquatic Preserve':'Estero Bay',
                'Biscayne Bay Aquatic Preserve':'Biscayne Bay','Matlacha Pass Aquatic Preserve':'Matlacha Pass',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island Sound'}
dictArea3    = {'Gasparilla Sound-Charlotte Harbor Aquatic Preserve':'ch','Big Bend Seagrasses Aquatic Preserve':'bb',
                'Guana Tolomato Matanzas National Estuarine Research Reserve':'gtm','Estero Bay Aquatic Preserve':'eb',
                'Biscayne Bay Aquatic Preserve':'bbay','Matlacha Pass Aquatic Preserve':'Matlacha Pass AP',
                'Lemon Bay Aquatic Preserve':'Lemon Bay','Cape Haze Aquatic Preserve':'Cape Haze','Pine Island Sound Aquatic Preserve':'Pine Island'}
dictPara     = {"Salinity":'S','Total Nitrogen':'TN','Dissolved Oxygen':'DO','Turbidity':'T','Secchi Depth':'SD'}
dictUnits    = {"Salinity":"ppt","Total Nitrogen": "mg/L","Dissolved Oxygen": "mg/L","Turbidity": "NTU", "Secchi Depth": "m"}
listArea     = ['Guana Tolomato Matanzas National Estuarine Research Reserve','Biscayne Bay Aquatic Preserve','Big Bend Seagrasses Aquatic Preserve','Cape Haze Aquatic Preserve',
               'Gasparilla Sound-Charlotte Harbor Aquatic Preserve', 'Pine Island Sound Aquatic Preserve', 'Matlacha Pass Aquatic Preserve', 'Lemon Bay Aquatic Preserve',
               'Estero Bay Aquatic Preserve']
listPara     = ["Salinity","Total Nitrogen","Dissolved Oxygen","Turbidity","Secchi Depth"]
SpatialRef   = '3086'

# 2. Run the Function to Generate Raster Files

In [7]:
for i in data.index:
    for para in listPara:
        name = [dictArea[data.iloc[i]["ma"]],dictArea3[data.iloc[i]["ma"]],data.iloc[i]["st_Year"],data.iloc[i]["season"],dictPara[para]]
        out_raster,out_ga_layer,ga_to_raster = misc.autointerpolation.interpolation_auto(method = "rk",
                           gis_path  = gis_path,                                                              
                           dataframe = dfAll,
                           managed_area = data.iloc[i]["ma"],
                           Year = data.iloc[i]["st_Year"],
                           Season = data.iloc[i]["season"],
                           start_date = data.iloc[i]["s_start"],
                           end_date = data.iloc[i]["s_end"],
                           parameter = para,
                           covariates = data.iloc[i]["covariates"].split("+"),
                           out_raster = name,
                           out_ga_layer = name,
                           predict_std_err = name)

['GTM Reserve', 'gtm', 2018, 'Summer', 'S']
Start the interpolation with the RK method
--- Time lapse: 82.1975347995758 seconds ---
['GTM Reserve', 'gtm', 2018, 'Summer', 'TN']
Start the interpolation with the RK method
--- Time lapse: 80.04730582237244 seconds ---
['GTM Reserve', 'gtm', 2018, 'Summer', 'DO']
Start the interpolation with the RK method
--- Time lapse: 90.90573143959045 seconds ---
['GTM Reserve', 'gtm', 2018, 'Summer', 'T']
Start the interpolation with the RK method
Turbidity in 2018 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

['GTM Reserve', 'gtm', 2018, 'Summer', 'SD']
Start the interpolation with the RK method
Secchi Depth in 2018 Summer caused an error:
ERROR 040039: Not enough data to compute method.
Failed to execute (EBKRegressionPrediction).

